<a href="https://colab.research.google.com/github/fabiomatricardi/ZephyrGPT4ALL/blob/main/Exploratory_Document_Analysis%E2%80%A6_is_this_a%C2%A0thing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/fabiomatricardi/LaminiChat/raw/main/Huggingfacehub.png" width=400>

## Test  MBZUAI/LaMini-Flan-T5-77M

In [1]:
%%capture
!pip install transformers
!pip install langchain
!pip install tiktoken

In [2]:
## Test  MBZUAI/LaMini-Flan-T5-77M
# to verify if smaller model give shorter inference speed but still good quality
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model77 = pipeline(model="MBZUAI/LaMini-Flan-T5-77M")
model77(prompt, max_length=128, repetition_penalty=1.1, do_sample=True)

[{'generated_text': "Subject: Alpaca Hello [Name], I hope this email finds you well. I wanted to reach out to you about the liar that I like, and you seem to have a preference for flan. As they are known for their powerful flavor and complex flavors, it's important for our company to prioritize their health and wellbeing. I appreciate your recommendation, and I am confident that I will be able to handle any issue that arises within your timeline. I would greatly appreciate it if you could update me on this topic. Looking forward to your feedback and assistance. Best regards, [Your Name"}]

In [3]:
from rich import print
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)

In [4]:
%%capture
!wget https://github.com/fabiomatricardi/ZephyrGPT4ALL/raw/main/EN_Vector%20Search%20Is%20Not%20All%20You%20Need%20by%20Anthony%20Alc.txt

In [5]:
with open('/content/EN_Vector Search Is Not All You Need by Anthony Alc.txt') as f:
  fulltext = f.read()
f.close()

Load the document and splitit into Chunks by TOKENS (450 tokens + 10 in overlap)

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=450, chunk_overlap=10)
sum_context = TOKENtext_splitter.split_text(fulltext)

Generate a Summary of the entire document

In [7]:
final = ''
strt = datetime.datetime.now()
for i in trange(0,len(sum_context)):
  text = sum_context[i]
  template_bullets = f'''ARTICLE: {text}

  What is a one-paragraph summary of the above article?

  '''
  start = datetime.datetime.now()
  res = model77(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  final = final + ' '+ res
  delta = datetime.datetime.now()-start
  console.print(f"[green2]Completed in {delta}")

delt = datetime.datetime.now()-strt
console.print(Markdown("# FULL SUMMARY"))
console.print(Markdown(final))
console.print(f"[red1 bold]Full summary Completed in {delt}")
console.print(Markdown("---"))

Output()

Completed in 0:00:02.433196

Completed in 0:00:01.540894

Completed in 0:00:01.189913

Completed in 0:00:02.470012

Completed in 0:00:01.715025

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                FULL SUMMARY                                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The article discusses the concept of vector search, which involves encoding data into vector representations  
and finding patterns in text data that can be encoded. It also highlights how it is essential for systems to  
produce human-like responses to queries by using neural generative models such as GPT-3. The article discusses
the limitations of vector search and how it is an active area of research to address issues related to        
linguistically distinct results, such as fragmentation in data, lack of relevant context, and difficulty      
finding meaningful answers. The article discusses how to use a multi-document question answering system,      
including the concept of complex reasoning and its importance in understanding the diversity of information   
used in different passage sizes. The article discusses the importance of using semantic similarity in a       
knowledge graph to find and analyze interconnected information, including time-ordered events, structure      
relationships, hierarchy links, hierarchical document structures, and implicit modeling. It also emphasizes   
how KGP provides a richer substrate for reasoning about interconnected information through its use of         
structured representations beyond just semantic similarity. AG systems employ hybrid approaches to solve      
problems such as vector search, graph-based knowledge representation, multi-step reasoning modules, and       
transparent ranking algorithms. By using a diverse toolkit of techniques, there is no single solution for     
robust retrieval in real-world question answering.

Full summary Completed in 0:00:09.397670

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Synoptic FUNCTIONS

A single function that takes Document Chunk Colletion and create:
- a general Summary
- a rewritten Summary in an eaasy to understand tone

In [10]:
def summarize_rewrite(sum_context):
  final = ''
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_bullets = f'''ARTICLE: {text}

    What is a one-paragraph summary of the above article?

    '''
    res = model77(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    final = final + ' '+ res

  ## REWRITE FUNCTION
  template_final = f'''Rewrite the following text in an easy to understand tone: {final}
  '''
  res = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  elaps = datetime.datetime.now() - strt
  console.print(Markdown("# SUMMARY and REWRITE"))
  console.print(Markdown(final))
  console.print("[green2 bold]---")
  console.print(Markdown(res))
  console.print(f"[red1 bold]Full RAG PREPROCESSING Completed in {elaps}")
  console.print(Markdown("---"))
  return final, res


In [11]:
f,r = summarize_rewrite(sum_context)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                            SUMMARY and REWRITE                                             ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The article discusses the revolutionizing open-domain question answering by introducing vector search, which  
involves finding relevant context passages using embedding models and neural generative modules. It also      
emphasizes that vector search has inherent constraints that can hinder the system's ability to reason in a    
nuanced manner. The article discusses the limitations of vector search and how it enables multilingual        
searches, but also explores alternative techniques that can help improve reasoning capabilities of large      
language models. It highlights the importance of understanding the strengths and weaknesses of current AI     
tools in creating efficient and accurate responses to questions. The article discusses the inherent trade-offs
between precision and tractability when choosing passage sizes, arguing that complex reasoning is essential   
for multi-hop reasoning. Some questions require synthesizing facts spread across multiple documents and are   
difficult to explain, verify, and improve. Vector search is opaque in standard vector search pipelines, but it
can be used to collect and contextualize information from different sources. In general, understanding        
semantic similarity requires modeling diverse relationships between content. The article provides a guide on  
how to use the concept of time-ordered events and concepts in KGP, including their relationship relationships,
structure relationships, interconnectedness, and semantic similarity. It also discusses how KGP is used for   
organizing information through hierarchical document structures that allow it to be linked chronologically    
without any surrounding context. The article discusses how using hybrid approaches to search and identify     
relevant information can help overcome weaknesses in the AG system. It suggests that leveraging diverse tools 
such as algorithms is crucial for robust retrieval for real-world question answering.

---

The article discusses the revolutionizing open-domain question answering by using vector search, which        
involves finding relevant context passages using embedding models and neural generative modules. It also      
highlights limitations of vector search and how it allows multilingual searches but explores alternative      
techniques that can improve reasoning capabilities of large language models. Understanding complex reasoning  
is essential for multi-hop reasoning because complexity is important in understanding semantic similarity. In 
general, understanding semantic similarity requires modeling diverse relationships between content.           
Additionally, KGP provides a guide on how to use time-ordered events and concepts in KGP, including their     
relationship relationships, structure relationships, interconnectedness, and semantic similarity. This helps  
overcome weaknesses in the AG system and suggests that leveraging different tools such as algorithms is       
crucial for robust retrieval for real-world question answering.

Full RAG PREPROCESSING Completed in 0:00:13.239432

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

A routine to have the LLM generate the questions from the Collection of Chunks of the document<br>
After that the RAG prompt is using the previously generated Summary `f`as context for the answers

In [12]:
# Generate Suggested questions from the text
# Then Reply to the questions
final = ''
strt = datetime.datetime.now()
for i in trange(0,len(sum_context)):
  text = sum_context[i]
  template_final = f'''{text}.\nAsk few question about this article.
'''
  res = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  final = final + '\n '+ res

delt = datetime.datetime.now()-strt
console.print(Markdown("---"))
console.print(f"[red1 bold]Questions generated in {delt}")
lst = final.split('\n')
final_lst = []
for items in lst:
  if items == '':
    pass
  else:
    final_lst.append(items)

for item in final_lst:
  question = item

  template_qna = f'''Read this and answer the question. If the question is unanswerable, ""say \"unanswerable\".\n\n{f}\n\n{question}
  '''

  start = datetime.datetime.now()
  res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  elaps = datetime.datetime.now() - start
  console.print(f"[bold deep_sky_blue1]{question}")
  console.print(Markdown(res))
  console.print(f"[red1 bold]Qna Completed in {elaps}")
  console.print(Markdown("---"))

Output()

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Questions generated in 0:00:03.595540

 What is the key advantage of vector search in RAG?

The key advantage of vector search in RAG is that it enables multilingual searches, but also explores         
alternative techniques that can help improve reasoning capabilities of large language models.

...

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

 What are some of the advantages and disadvantages that can be achieved using hybrid approaches in AG systems?

Advantages and disadvantages that can be achieved using hybrid approaches in AG systems include: - The        
advantages of vector search are inherent constraints that hinder the system's ability to reason in a nuanced  
manner. - Vector search has limitations which may hinder the system's ability to reason in a nuanced manner. -
The limitations of vector search have inherent constraints that can hinder the system's ability to reason in a
nuanced manner. - Vector search is opaque in standard vector search pipelines, but it can be used to collect  
and contextualize information from different sources.

Qna Completed in 0:00:03.669422

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Finally a Function that does all of the above


In [14]:
def summarize_rewrite(sum_context):
  final = ''
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_bullets = f'''ARTICLE: {text}

    What is a one-paragraph summary of the above article?

    '''
    res = model77(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    final = final + ' '+ res

  ## REWRITE FUNCTION
  template_final = f'''Rewrite the following text in and easy to understand tone: {final}
  '''
  rew = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  elaps = datetime.datetime.now() - strt
  console.print(Markdown("# SUMMARY and REWRITE"))
  console.print(Markdown(final))
  console.print("[green2 bold]---")
  console.print(Markdown(rew))
  console.print(f"[red1 bold]Full RAG PREPROCESSING Completed in {elaps}")
  console.print(Markdown("---"))

  # Generate Suggested questions from the text
  # Then Reply to the questions
  finalqna = ''
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_final = f'''{text}.\nAsk few question about this article.
  '''
    res = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    finalqna = finalqna + '\n '+ res

  delt = datetime.datetime.now()-strt
  console.print(Markdown("---"))
  console.print(f"[red1 bold]Questions generated in {delt}")
  lst = finalqna.split('\n')
  final_lst = []
  for items in lst:
    if items == '':
      pass
    else:
      final_lst.append(items)

  qna = []
  for item in final_lst:
    question = item

    template_qna = f'''Read this and answer the question. If the question is unanswerable, ""say \"unanswerable\".\n\n{f}\n\n{question}
    '''

    start = datetime.datetime.now()
    res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    elaps = datetime.datetime.now() - start
    console.print(f"[bold deep_sky_blue1]{question}")
    console.print(Markdown(res))
    console.print(f"[red1 bold]Qna Completed in {elaps}")
    console.print(Markdown("---"))
    qna.append({'question':item,
                'answer': res})

  return final, rew, qna

In [15]:
f1,r1,q1 = summarize_rewrite(sum_context)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                            SUMMARY and REWRITE                                             ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

The article discusses the revolutionization of open-domain question answering, which involves encoding data   
into vector representations and finding vectors similar to the search query. It also highlights that this     
approach has inherent constraints that can hinder the system's ability to reason in a nuanced manner. The     
article discusses the limitations of vector search and its exploration of how it can be used to capture       
diverse relationships and interconnections between documents. It also highlights the importance of            
understanding the strengths and weaknesses of AI tools, including knowledge graph prompting, which helps in   
identifying potential gaps in relevant information. The article discusses the importance of multi-hop         
reasoning in determining passage sizes, which involves manipulating and matching information across multiple  
documents. It also highlights how complex questions require synthesizing facts spread across multiple document
formats. The article provides a summary of the benefits and disadvantages of using knowledge graphs in RAG    
systems, including its ability to connect information between different types of data, semantic relationships,
and interconnectedness. The article discusses the use of hybrid approaches to solve problems related to vector
search, graph-based knowledge representation, multi-step reasoning modules, and transparent ranking algorithms
in AG systems. It also highlights that there is no single solution for robust retrieval for real-world        
question answering.

---

The article discusses the revolutionization of open-domain question answering, which involves encoding data   
into vector representations and finding vectors similar to search query. It also highlights the limitations of
vector search and its exploration of how it can be used to capture diverse relationships and interconnections 
between documents. Additionally, knowledge graph prompting helps in understanding potential gaps in relevant  
information. Understanding multi-hop reasoning is crucial for identifying key insights from multiple document 
formats. Knowledge graphs are commonly used in RAG systems, but there are no single solution for robust       
retrieval for real-world question answering.

...

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

 What are the advantages of using hybrid approaches to solve problems in AG systems?

Advantages of using hybrid approaches to search and identify relevant information can help overcome weaknesses
in the AG system.

Qna Completed in 0:00:00.707043

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [16]:
console.print(q1)

[
    {
        'question': ' What is the main key step in retrieval Augmented Generation (RAG)?',
        'answer': 'The main key step in retrieval Augmented Generation (RAG) is to identify relevant 
information through leveraging diverse tools such as algorithms.'
    },
    {
        'question': ' What is the main issue in vector search systems and how does it relate to linguistically
distinct results?',
        'answer': 'The main issue in vector search systems is the limitations of vector search and how it 
enables multilingual searches. It relates to linguistically distinct results by understanding the strengths 
and weaknesses of current AI tools in creating efficient and accurate responses to questions.'
    },
    {
        'question': ' What are some of the key differences between precision and tractability when choosing 
passage sizes?',
        'answer': 'Precision is essential for multi-hop reasoning.'
    },
    {
        'question': ' What are the main advantages of using KGP in relation to structure relationships?',
        'answer': 'The main advantages of using KGP in relation to structure relationships are their 
relationship relationships, structure relationships, interconnectedness, and semantic similarity.'
    },
    {
        'question': ' What are the advantages of using hybrid approaches to solve problems in AG systems?',
        'answer': 'Advantages of using hybrid approaches to search and identify relevant information can help 
overcome weaknesses in the AG system.'
    }
]



---



---



---



---



## Test  MBZUAI/LaMini-Flan-T5-248M

In [ ]:
%%capture
!pip install transformers
!pip install langchain
!pip install tiktoken

In [ ]:
## Test  MBZUAI/LaMini-Flan-T5-248M
# to verify if smaller model give shorter inference speed but still good quality
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model248 = pipeline(model="MBZUAI/LaMini-Flan-T5-248M")
model248(prompt, temperature=0.2, max_length=128, repetition_penalty=1.1, do_sample=True)

[{'generated_text': "Subject: Alpaca with Flan Dear [Recipient], I hope this email finds you well. I am writing to inquire about an alpaca that likes flan. I have been searching for a few years now and I have found a great one. I would love to learn more about this animal and its preferences. Could you please provide me with more information about the alpaca's breed, size, and health? Additionally, I would appreciate any recommendations or information you can provide on how to care for it. Thank you for your time and consideration. I look forward to hearing back from you"}]

In [ ]:
from rich import print
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)

In [ ]:
with open('/content/EN_Vector Search Is Not All You Need by Anthony Alc.txt') as f:
  fulltext = f.read()
f.close()

In [ ]:
text = fulltext[:2000]

In [ ]:
# Test one-paraghraph summary with Lamini77M  2000 charachters
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model248(template_bullets, temperature=0.2, repetition_penalty=1.1, max_length=400, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

The article discusses the advantages of using hybrid approaches to improve the performance of AG systems,     
including vector search, graph-based knowledge representation, multi-step reasoning modules, and transparent  
ranking algorithms. It emphasizes that there is no single solution, but a diverse toolkit of techniques is    
essential for robust retrieval for real-world question answering.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:05.716257

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=450, chunk_overlap=10)
sum_context = TOKENtext_splitter.split_text(fulltext)

In [ ]:
text = sum_context[0]
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model248(template_bullets, temperature=0.2, repetition_penalty=1.1, max_length=400, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

The article discusses the importance of retrieval augmented generation (RAG) in enabling systems to produce   
human-like responses to open-domain questions. However, it also highlights the limitations of this approach,  
particularly when dealing with complex multi-hop reasoning across multiple documents. The article explains how
vector search is used to search for information using vector representations of data, and how it works by     
encoding data into vectors using embedding models and using vector similarity metrics to identify vectors with
the smallest distance.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:13.809592

In [ ]:
final = ''
strt = datetime.datetime.now()
for i in trange(0,len(sum_context)):
  text = sum_context[i]
  template_bullets = f'''ARTICLE: {text}

  What is a one-paragraph summary of the above article?

  '''
  start = datetime.datetime.now()
  res = model248(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  final = final + ' '+ res
  delta = datetime.datetime.now()-start
  console.print(f"[green1]Completed in {delta}")

delt = datetime.datetime.now()-strt
console.print(Markdown(final))
console.print(Markdown("---"))
console.print(f"[red1 bold]Full summary Completed in {delt}")

Output()

Completed in 0:00:10.938991

Completed in 0:00:08.849567

Completed in 0:00:15.015130

Completed in 0:00:05.478693

The article discusses the limitations of retrieval augmented generation (RAG) and how it can hinder the       
system's ability to reason in a nuanced manner. It emphasizes that vector search is not all about finding     
relevant information, but rather using vector representations of data for complex multi-hop reasoning across  
multiple documents. The article discusses the limitations of vector search and how it falls short in capturing
diverse relationships between documents, as well as exploring alternative techniques such as knowledge graph  
prompting. It also highlights the semantic gap between questions and answers and how one embedding vector can 
be used to identify relevant sections from irrelevant ones. The article discusses the trade-offs between      
precision and tractability when choosing passage sizes, as well as the challenges faced by vector search in   
multi-hop reasoning. It also mentions that simple similarity search is ill-equipped for such tasks, and it    
struggles to collect and contextualize facts from different passages. Additionally, the article highlights the
importance of modeling diverse relationships between content through knowledge graphs, which can be used for  
multiple document question answering (MD-QA). The article discusses different methods for analyzing and       
understanding interconnected information, including time-ordered events, entity relationships, structure      
relationships, knowledge graphs, and vector search. It emphasizes the importance of using these approaches to 
extract meaningful insights from data and how they can be used in various applications such as RAG systems.   
The article discusses the advantages of using hybrid approaches to improve the robust retrieval of real-world 
questions in AG systems. It suggests that combining vector search, graph-based knowledge representation,      
multi-step reasoning modules, and transparent ranking algorithms can help overcome weaknesses in these        
systems.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Full summary Completed in 0:00:51.748538

In [ ]:
final = ''
strt = datetime.datetime.now()
for i in trange(0,len(sum_context)):
  text = sum_context[i]
  template_bullets = f'''TEXT: {text}

What are the most important parts of this text?

'''
  start = datetime.datetime.now()
  res = model(template_bullets, repetition_penalty=1.1, max_length=300, do_sample=True)[0]['generated_text']
  final = final + ' '+ res
  delta = datetime.datetime.now()-start
  console.print(f"[blue]Completed in {delta}")

delt = datetime.datetime.now()-strt
console.print(Markdown(final))
console.print(Markdown("---"))
console.print(f"[red1 bold]Full summary Completed in {delt}")

Output()

Completed in 0:00:04.311568

Completed in 0:00:05.912059

Completed in 0:00:07.880602

Completed in 0:00:08.859415

Completed in 0:00:04.225252

The most important parts of this text are: 1. Understanding Vector Search Is Not All You Need 2. Encoding data
into vectors 3. Developing tools for deep learning 4. Retrieval Augmented Generation (RAG) Some important     
parts of this text are the limitations of vector search, transforming data into vectors and searching in      
high-dimensional semantic space, understanding the strengths and weaknesses of the right-hand side of it,     
using alternative techniques like knowledge graph prompting or knowledge graph prompting to overcome these    
shortcomings, and analyzing the strengths and weaknesses of existing AI tools. The text discusses issues      
related to comparative study on spatial relationships, linguistics, and communication across multiple         
documents, from precision to tractability to accuracy. Some questions or questions require synthesizing facts 
spread across multiple documents. Vector search is ill-equipped for complex multi-hop reasoning, but complex  
semantic similarity search is hindering the overall understanding of what constitutes significant difference  
in representation between content, and in general, generalization involves modeling diverse relationships     
between content. The text also mentions the use of comprehensive comparisons to make comparisons to different 
sources and discussing historical topics. The most important parts of this text are: 1. Time-ordered events   
are linked chronologically. 2. Coreference links are added between passages referencing the same real-world   
entities which allows entity-centric reasoning. 3. Structural relationships allow contextual categorization   
and object search. 4. Temporal relationships are not directly modeled by standard vector search. 5. Ordering  
passages chronologically based on the events they describe enables reasoning about unfolding narratives and   
timelines. 6. Vector similarity scores do not factor temporal dynamics in any way. 7. Predictable knowledge   
around entities is lost in sentence embeddings. 8. Vector search protects the signal and ensures structural   
advantages when aggregating facts about specific entities across documents. 9. Vector search empowers         
efficient approximate matching based on semantic similarity. 10. Reinforcement generalization includes        
hierarchy and coherence. The most important parts of the text are: - Subject: AG systems using hybrid         
approaches for vector search with graph-based knowledge representation, multi-step reasoning modules, and     
transparent ranking algorithms - Methods used to use hybrid approaches to tackle these weaknesses and help    
overcome them - Definition of the methodology for robust retrieval as always - Advantages of employing smart  
algorithms that are diverse: adaptability, agility, and robustness - Resilience.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Full summary Completed in 0:00:31.267395

## Test flan-alpaca-large  T5 model

In [ ]:
%%capture
!pip install transformers
!pip install langchain
!pip install tiktoken

In [ ]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model = pipeline(model="declare-lab/flan-alpaca-large")
model(prompt, max_length=128, do_sample=True)

[{'generated_text': "Dear [Name], I am writing to tell you about my alpaca, Mr. Flan. My alpaca loves flan. He is always inquisitive and loves to be in a pleasant environment. He is an incredibly loyal and gentle alpaca and loves to be petted. He is a loyal and friendly pet, who loves playing fetch, cuddling with his owner's back and tossing frilly tails. He is a great pet who loves to be around people and make everyone smile. I am writing to tell you that my alpaca is very"}]

In [ ]:
from rich import print
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)

In [ ]:
with open('/content/EN_Vector Search Is Not All You Need by Anthony Alc.txt') as f:
  fulltext = f.read()
f.close()

In [ ]:
text = fulltext[:2000]

In [ ]:
# Test of bullet pooins 4bit wizard-orca-3b  2000 charachters
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model(template_bullets, max_length=400, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

The vector search paradigm is becoming increasingly popular in AI-driven solutions, but it presents many      
challenges. The most important step in retrieving context passages is encoding them into numeric vector       
representations. A search for a given type of dataset involves utilizing the same features such as keywords   
and descriptions on each file in order to ensure an efficient retrieval process. To achieve success, this     
should involve analyzing the data from various angles. Additionally, when the search is supervised, there     
should be robust support for decision-making and an understanding of the relevant context in the data.        
Finally, with a complex multi-hop reasoning approach, there should be a strong emphasis on exploring further  
the context and the context's relationships.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:40.394970

In [ ]:
text = fulltext[2000:4200]
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model(template_bullets, max_length=400, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

This article examines the issues of vector search, its strengths and weaknesses, and its potential            
applications in AI and domains such as natural language processing. It also discusses the implications of     
these tradeoffs and strategies for overcoming them. Ultimately, the paper will help to provide clear framework
for understanding vector search and exploring alternatives, such as Knowledge Graph prompting, that can help  
to enhance the reasoning capabilities of large AI systems.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:27.757450

In [ ]:
text = fulltext[4200:]
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model(template_bullets, max_length=400, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

Standard vector search is ill-equipped for multi-hop reasoning due to its opaqueness and lack of transparency.
Knowledge graph has several advantages including: (1) enabling reasoning about structured relationships       
between passages; (2) combining semantic similarity and structure relationships between documents; (3)        
allowing a higher level of context manipulation; and (4) allowing an individual-centric approach to query     
solving.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:44.812739

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=10)
sum_context = TOKENtext_splitter.split_text(fulltext)

In [ ]:
text = sum_context[0]
template_bullets = f'''ARTICLE: {text}

What is a one-paragraph summary of the above article?

'''
start = datetime.datetime.now()
res = model(template_bullets, max_length=300, do_sample=True)[0]['generated_text']
delta = datetime.datetime.now()-start
console.print(Markdown(res))
console.print(Markdown("---"))
console.print(f"[red1 bold]Completed in {delta}")

Retrieval Augmented Generation (RAG) has revolutionized open-domain question answering, yet it has serious    
limitations. One of the most important components, the vector search over embedded passages, poses difficulty 
in a variety of complex context questions. Typically these questions involve multi-hop rationales, so a vector
search is necessary in order to find the best results. This enables a more complete response from multiple    
texts and is a powerful tool for question answering systems. However, the vector search has its limitations,  
such as the need to search in high-dimensional semantic space and be able to differentiate between different  
types of keywords, requiring further exploration.

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Completed in 0:00:41.833957